## first extract transcript


In [1]:
%load_ext autotime

time: 247 µs (started: 2024-07-31 14:49:32 +05:45)


In [1]:

import pickle

# Load the object from the file
with open('my_variable.pkl', 'rb') as file:
    transcript = pickle.load(file)

In [2]:
new_t = []
for item in transcript.words:
    new_t.append({ 'word': item['word'], 'round' : round(item['start'], 2), 'start' : item['start'], 'end': item['end']})

In [3]:
new_t

[{'word': 'Boom',
  'round': 1.8,
  'start': 1.7999999523162842,
  'end': 2.2799999713897705},
 {'word': 'All',
  'round': 2.28,
  'start': 2.2799999713897705,
  'end': 2.640000104904175},
 {'word': 'right',
  'round': 2.64,
  'start': 2.640000104904175,
  'end': 2.7200000286102295},
 {'word': "We're",
  'round': 2.8,
  'start': 2.799999952316284,
  'end': 2.9800000190734863},
 {'word': 'live',
  'round': 2.98,
  'start': 2.9800000190734863,
  'end': 3.259999990463257},
 {'word': 'Thank',
  'round': 3.48,
  'start': 3.4800000190734863,
  'end': 3.859999895095825},
 {'word': 'you',
  'round': 3.86,
  'start': 3.859999895095825,
  'end': 3.940000057220459},
 {'word': 'very',
  'round': 3.94,
  'start': 3.940000057220459,
  'end': 4.119999885559082},
 {'word': 'much',
  'round': 4.12,
  'start': 4.119999885559082,
  'end': 4.179999828338623},
 {'word': 'for',
  'round': 4.18,
  'start': 4.179999828338623,
  'end': 4.440000057220459},
 {'word': 'doing',
  'round': 4.44,
  'start': 4.440000

## divide transcript into 90 sec intervals


In [4]:


# Initialize variables
two_minutes = 90  # 2 minutes in seconds
chunks = []
current_chunk = []
start_time = new_t[0]['start']  # Start time of the first word

# Iterate through the list
for item in new_t:
    word, timestamp = item['word'], item['round']
    
    if timestamp - start_time < two_minutes:
        current_chunk.append((word, timestamp))
    else:
        chunks.append(current_chunk)
        current_chunk = [(word, timestamp)]
        start_time = timestamp

# Add the last chunk if it has words
if current_chunk:
    chunks.append(current_chunk)

# Unpack chunks into individual variables
chunk1, chunk2, chunk3 = chunks[:3]  # Adjust the number of chunks based on your need




# Note: This will print only the first 3 chunks. Adjust the code if you need more or fewer chunks.


In [5]:
len(chunks)

14

## extract the catchy parts

In [6]:
import asyncio
from openai import AsyncOpenAI
import time
import ast

client = AsyncOpenAI()

async def extract(prev, curr, nextt):
    
    
    output_format = """{
    "topic" : answer
    "start": start_time_in_seconds,
    "end" : end_time_in_seconds
    }"""
    
    input_prompt = f"""

    
    You are given an audio transcript in this format [('hello', 0), ('my', 1), ('name', 2)]

    the words are in consecutive order. The first index of each tuple contains the actual word being spoken and
    second index inside each tuple contains the timestamp at which the word is being spoken.
    
    Your task is to identify interesting, catchy topic being discussed in the transcript and give me the start and end timestamp
    for that topic and the topic name. Remember the duration of the topic being discussed should be between 60-90 seconds.
    
    The start timestamps should be catchy and should build up the initial words for our main topic that comes afterwards.
    The end timestamp should stop as soon as the topic ends, or when it starts to become verbose.
    
    You'll be given 3 values.
    Previous (list) : The transcript of previous timeframe.
    Current (list): The transcript of current timeframe.
    Next (list) : The transcript of next timeframe.
    
    Mainly, you should focus on "Current" transcript but it may contain truncated informations for which you can use
    the "Previous" and "Next" transcript.
    
    If the topic being discussed is incomplete then please add none to all the keys in the final output JSON
    
    Your output shouldf be in this JSON format
    
    {output_format}
    


    ### Input:
    
    Previous : {prev}
    Current : {curr}
    Next : {nextt}
    

    ### OUTPUT: 

    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent that identifies interesting topic being discussed 
        in the text."""},
        {"role": "user", "content": input_prompt},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content



In [7]:

async def generate_batch(chunks):

    tasks = []
    end = (int(len(chunks)/3)*3)
    for ind in range(0,end, 3):
        print(ind)

        prev = chunks[ind]
        curr = chunks[ind+1]
        nextt = chunks[ind+2]

        tasks.append(extract(prev,curr,nextt))


    results = await asyncio.gather(*tasks)
    return results



In [8]:
result = await generate_batch(chunks)

0
3
6
9


In [9]:
new_result = []
for item in result:
    stripped_item = item.strip('```')
    stripped_item = stripped_item.strip('json')
    new_result.append(ast.literal_eval(stripped_item))

In [10]:
new_result

[{'topic': "Life's Arc from Young Age to Philosopher",
  'start': 92.42,
  'end': 176.78},
 {'topic': 'Reading Habits and Information Consumption',
  'start': 362,
  'end': 480},
 {'topic': 'Celebrity Challenges and Anonymity',
  'start': 633.38,
  'end': 676.82},
 {'topic': 'Achieving Happiness and Inner Peace',
  'start': 903.76,
  'end': 1057.98}]

In [12]:
chunks[7]

[('is', 633.38),
 ('not', 634.44),
 ('healthy', 634.66),
 ('This', 635.12),
 ('is', 635.26),
 ('not', 635.34),
 ('a', 635.48),
 ('smart', 635.68),
 ('way', 635.86),
 ('to', 636.22),
 ('interact', 636.72),
 ('with', 637.02),
 ('people', 637.4),
 ('and', 637.58),
 ('that', 638.18),
 ('people', 638.44),
 ('have', 638.6),
 ('this', 638.94),
 ('weird', 639.24),
 ('challenge', 640.0),
 ('this', 640.44),
 ('weird', 640.82),
 ('thing', 641.02),
 ('that', 641.26),
 ('if', 641.68),
 ('you', 641.86),
 ('become', 642.28),
 ('famous', 642.58),
 ("there's", 642.98),
 ('this', 643.16),
 ('weird', 643.42),
 ('challenge', 643.62),
 ('where', 644.02),
 ('people', 644.24),
 ('just', 644.56),
 ('want', 645.0),
 ('to', 645.24),
 ('come', 645.38),
 ('to', 645.56),
 ('you', 645.72),
 ('especially', 645.9),
 ('today', 646.1),
 ('because', 646.5),
 ('if', 646.92),
 ('they', 647.06),
 ('can', 647.12),
 ('get', 647.24),
 ('a', 647.28),
 ('photo', 647.68),
 ('of', 647.68),
 ('you', 647.98),
 ('then', 648.52),
 ('

In [14]:
(90*3) / 60 

4.5

In [15]:
for index in range(len(new_result)):
    new_result[index]['transcript'] = []
    

for index, each_result in enumerate(new_result):
    for ind, item in enumerate(new_t):
        if item['round'] >= each_result['start'] and item['round'] <= each_result['end']:
            new_result[index]['transcript'].append(item)

new_result            
  

[{'topic': 'The Arc of Life and the Importance of Trying New Things',
  'start': 94.58,
  'end': 175.92,
  'transcript': [{'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552734,
    'end': 96.05999755859375},
   {'word': 'model',
    'round': 96.06,
    'start': 96.05999755859375,
    'end': 96.33999633789062},
   {'word': 'of',
    'round': 96.34,
    'start': 96.33999633789062,
    'end': 96.87999725341797},
   {'word': 'life',
    'round': 96.88,
    'start': 96.87999725341797,
    'end': 96.95999908447266},
   {'word': 'that',
    'round': 96.96,
    'start': 96.95999908447266,
    'end': 97.22000122070312},
   {'word': 'the',
    'round': 97.22,
    'start': 97.220001220

time: 97.8 ms (started: 2024-07-31 16:11:12 +05:45)


## combine it for every 5 seconds

In [16]:
# Function to combine words in 5-second intervals
def combine_words(transcript, interval=4):
    combined_transcript = []
    transcript.sort(key=lambda x: x['start'])
    
    start_time = transcript[0]['start']
    end_time = start_time + interval
    
    current_chunk = {'start': start_time, 'end': end_time, 'words': []}
    
    for item in transcript:
        if item['start'] >= end_time:
            combined_transcript.append(current_chunk)
            start_time = item['start']
            end_time = start_time + interval
            current_chunk = {'start': start_time, 'end': end_time, 'words': []}
        
        current_chunk['words'].append(item['word'])
    
    combined_transcript.append(current_chunk)  # Append the last chunk
    
    return combined_transcript



time: 2.59 ms (started: 2024-07-31 16:11:21 +05:45)


In [17]:
interval_words = (combine_words(new_result[0]['transcript']))

time: 563 µs (started: 2024-07-31 16:11:26 +05:45)


In [18]:
input_dict = {num: " ".join(item['words']) for num,item in enumerate(combine_words(new_result[0]['transcript']))}

input_dict

{0: 'So I like the model of life that the ancients had the Greeks',
 1: "the Romans right where you would start out and when you're young you're",
 2: "just like going to school Then you're going to war Then you're running a",
 3: "business Then you're supposed to serve in the Senate or the government Then you become a philosopher",
 4: "There's sort of this arc to life where you try your hand at everything",
 5: 'And as one of my friends says specialization is for insects right',
 6: "So everyone should just be able to do everything And so I don't believe in this model anymore",
 7: "of trying to focus your life down on one thing You've got one life Just",
 8: "do everything you're going to do I couldn't agree more And I",
 9: 'think that sometimes people find certain success in whatever the endeavor is',
 10: 'and then they think that that is their niche and they stick with it and they never change',
 11: "And they're almost out of fear Well it's hard because there's",
 12: 'an analo

time: 3.33 ms (started: 2024-07-31 16:11:29 +05:45)


In [21]:

async def identify_images(input_dict):
    
    
    output_format = """{
    "start": start_time_in_seconds, // edited start time
    "end" : end_time_in_seconds, //edited end time
    }"""
    
    input_prompt = """ You are given a dictionary like this 
    
    {'1': "Greeks the Romans right where you would start out and" , '2' : "when you're young you're just like going to school Then you're going" }
    
    your task is to find out the objects,things,place  that is being discussed for current key and make stable diffusion prompt using that topic.
    
    
    For example:
    If the text is : "Greeks the Romans right where you would start out and"
    
    The text can have meaningful keyword or topic like: "Ancient Romans"
    
    So your task is to formulate a short and precise stable diffusion prompt for the for each of keys the dictionary and output it in JSON format
    
    like this:
    
    
    {'1': "Ancient Romans: Majestic Roman colosseum, legionnaires in armor, classical architecture, togas, and laurel wreaths, set against a sunlit sky." , '2' : "School Boy: A young boy in a school uniform, carrying a backpack, holding books, standing in front of a school building with a cheerful expression, under a bright morning sky." }
    

    """

    input_variable = f"""
    
    ### Input Dictionary:
    
    {input_dict}
    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent responsible for constructing meaningful stable diffusion prompts from a given text"""},
        {"role": "user", "content": input_prompt +  input_variable},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content


time: 3.88 ms (started: 2024-07-31 16:15:47 +05:45)


In [22]:

async def identify_images_new(input_dict):
    
    
    output_format = """{
    "start": start_time_in_seconds, // edited start time
    "end" : end_time_in_seconds, //edited end time
    }"""
    
    input_prompt = """ You are given continuous list of dictionary, which are actually the transcript from a video
    For example:
    
    {'1': "Greeks the Romans right where you would start out and" , '2' : "when you're young you're just like going to school Then you're going" }
    your task is to find out which keys in the dictionary can be represented into an image and write a stable diffusion 
    prompt for those keys.
    
    For example:
    Key 1 can be represented into an image of Ancient romans with greeks.
    
    For the keys having no particular meaning please output None for those keys.
    
    formulate a short and precise stable diffusion prompt for the for each of keys the dictionary and output it in JSON format
    
    So an example output for the input above can look like this.
    like this:
    
    
    {'1': "Ancient Romans: Majestic Roman colosseum, legionnaires in armor, classical architecture, togas, and laurel wreaths, set against a sunlit sky." , '2' : "School Boy: A young boy in a school uniform, carrying a backpack, holding books, standing in front of a school building with a cheerful expression, under a bright morning sky." }
    

    """

    input_variable = f"""
    
    ### Input Dictionary:
    
    {input_dict}
    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent responsible for constructing meaningful stable diffusion prompts from a given text"""},
        {"role": "user", "content": input_prompt +  input_variable},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content


time: 3.99 ms (started: 2024-07-31 16:15:50 +05:45)


In [23]:
ans = await identify_images(input_dict)

time: 8.15 s (started: 2024-07-31 16:15:51 +05:45)


In [24]:
import json

ans = json.loads(ans)

time: 892 µs (started: 2024-07-31 16:16:01 +05:45)


In [25]:
input_dict

{0: 'So I like the model of life that the ancients had the Greeks',
 1: "the Romans right where you would start out and when you're young you're",
 2: "just like going to school Then you're going to war Then you're running a",
 3: "business Then you're supposed to serve in the Senate or the government Then you become a philosopher",
 4: "There's sort of this arc to life where you try your hand at everything",
 5: 'And as one of my friends says specialization is for insects right',
 6: "So everyone should just be able to do everything And so I don't believe in this model anymore",
 7: "of trying to focus your life down on one thing You've got one life Just",
 8: "do everything you're going to do I couldn't agree more And I",
 9: 'think that sometimes people find certain success in whatever the endeavor is',
 10: 'and then they think that that is their niche and they stick with it and they never change',
 11: "And they're almost out of fear Well it's hard because there's",
 12: 'an analo

time: 2.58 ms (started: 2024-07-31 16:16:01 +05:45)


In [26]:
for index, item in enumerate(interval_words):
    print(index,ans.get(str(index), None) )
    interval_words[index]['keywords'] = ans.get(str(index), None)

0 Ancient Greeks: Bustling marketplace in ancient Athens, philosophers debating under olive trees, marble statues of gods and goddesses, and athletes competing in the original Olympic games.
1 Ancient Romans: Majestic Roman colosseum, legionnaires in armor, classical architecture, togas, and laurel wreaths, set against a sunlit sky.
2 School Days: Brightly colored classrooms filled with eager students, teachers at the blackboard, school bags and lunchboxes scattered about, and the sound of laughter echoing in the hallways.
3 Government Service: Stately government buildings, politicians in formal attire debating in ornate chambers, laws being written on parchment scrolls, and the weight of responsibility etched on faces.
4 Life's Journey: A winding path through varied landscapes, passing through stages of learning, battle, commerce, leadership, and contemplation, each experience shaping the traveler.
5 Versatile Living: A bustling cityscape where people wear many hats, switching roles e

In [30]:
interval_words

[{'start': 94.58000183105469,
  'end': 98.58000183105469,
  'words': ['So',
   'I',
   'like',
   'the',
   'model',
   'of',
   'life',
   'that',
   'the',
   'ancients',
   'had',
   'the',
   'Greeks'],
  'keywords': 'Ancient Greeks: Bustling marketplace in ancient Athens, philosophers debating under olive trees, marble statues of gods and goddesses, and athletes competing in the original Olympic games.'},
 {'start': 98.76000213623047,
  'end': 102.76000213623047,
  'words': ['the',
   'Romans',
   'right',
   'where',
   'you',
   'would',
   'start',
   'out',
   'and',
   'when',
   "you're",
   'young',
   "you're"],
  'keywords': 'Ancient Romans: Majestic Roman colosseum, legionnaires in armor, classical architecture, togas, and laurel wreaths, set against a sunlit sky.'},
 {'start': 102.80000305175781,
  'end': 106.80000305175781,
  'words': ['just',
   'like',
   'going',
   'to',
   'school',
   'Then',
   "you're",
   'going',
   'to',
   'war',
   'Then',
   "you're",
   

time: 12.8 ms (started: 2024-07-31 16:34:34 +05:45)


## find index to add images

In [31]:
len(interval_words)

20

time: 1.76 ms (started: 2024-07-31 16:34:37 +05:45)


In [32]:
int(18*0.2)

3

time: 2.74 ms (started: 2024-07-31 16:34:37 +05:45)


## new algo

In [41]:
for index in range(1, 21-3, 3):
    
    print('index',index, index+1 ,index + 3 - 1 )
    number = random.randint(index +1, index + 3 -1)
    print(number)

index 1 2 3
2
index 4 5 6
6
index 7 8 9
8
index 10 11 12
11
index 13 14 15
14
index 16 17 18
18


In [24]:
19*5

95

In [15]:
import random

index_to_find_images = []


for index in range(0+1, len(interval_words)-2, 3):
    print('index',index, index + 1 ,index + 3 - 1 )
    number = random.randint(index + 1, index + 3 - 1)
    print(number)
    
    if interval_words[number]['keywords'] is not None:
        index_to_find_images.append({number: interval_words[number]})


NameError: name 'interval_words' is not defined

In [34]:
len(interval_words)

20

time: 3.74 ms (started: 2024-07-31 16:34:45 +05:45)


In [35]:
import random

index_to_find_images = []


for index in range(0+1, len(interval_words)-3, 4):
    print('index',index, index + 1 ,index + 3 - 1 )
    number = random.randint(index + 1, index + 3 - 1)
    print(number)
    
    if interval_words[number]['keywords'] is not None:
        index_to_find_images.append({number: interval_words[number]})


index 1 2 3
3
index 5 6 7
7
index 9 10 11
11
index 13 14 15
14
time: 1.05 ms (started: 2024-07-31 16:34:47 +05:45)


In [36]:
index_to_find_images

[{3: {'start': 106.9000015258789,
   'end': 110.9000015258789,
   'words': ['business',
    'Then',
    "you're",
    'supposed',
    'to',
    'serve',
    'in',
    'the',
    'Senate',
    'or',
    'the',
    'government',
    'Then',
    'you',
    'become',
    'a',
    'philosopher'],
   'keywords': 'Government Service: Stately government buildings, politicians in formal attire debating in ornate chambers, laws being written on parchment scrolls, and the weight of responsibility etched on faces.'}},
 {7: {'start': 124.94000244140625,
   'end': 128.94000244140625,
   'words': ['of',
    'trying',
    'to',
    'focus',
    'your',
    'life',
    'down',
    'on',
    'one',
    'thing',
    "You've",
    'got',
    'one',
    'life',
    'Just'],
   'keywords': "Holistic Living: Embracing the richness of life's experiences, refusing to confine oneself to a single path, exploring all facets of existence with boundless curiosity and vigor."}},
 {11: {'start': 141.9199981689453,
  

time: 5.76 ms (started: 2024-07-31 16:34:55 +05:45)


## get image

In [37]:
import aiohttp
import asyncio
from PIL import Image
from io import BytesIO

async def fetch_image_id(session, keyword):
    url = f"https://api.pexels.com/v1/search?query={keyword}&per_page=1&orientation=portrait&size=small"
    headers = {
        "Authorization": "DS6m37FqnoQRq8JKFy7J0hnmg1yqQvzImxNHRGjhfLgE8qsQtgDi11U9"
    }

    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            # Parse the response JSON asynchronously
            data = await response.json()
            photo_url = data['photos'][0]['src']['portrait']

            # Make a request to get the photo
            async with session.get(photo_url) as photo_response:
                if photo_response.status == 200:
                    # Read the image content
                    photo_data = await photo_response.read()

                    # Open the image
                    image = Image.open(BytesIO(photo_data))

                    # Save the image
                    image.save(f"images/{keyword}.jpeg")
                    print(f"Image for keyword '{keyword}' saved as '{keyword}.jpeg'")
                else:
                    print(f"Failed to retrieve the photo: {photo_response.status}")
        else:
            print(f"Failed to retrieve data for {keyword}: {response.status}")

async def get_image_ids(keywords):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image_id(session, keyword) for keyword in keywords]
        await asyncio.gather(*tasks)


time: 531 ms (started: 2024-07-31 16:35:11 +05:45)


In [38]:
keywords = []
for item in index_to_find_images:
    for _,val in item.items():
        keywords.append(val['keywords'])
        
final_images = await get_image_ids(keywords)

Image for keyword 'Unfinished Journey: A vista of uncharted territories, the allure of unconquered summits beckoning the daring, the realization that the journey is ongoing, with the destination forever elusive.' saved as 'Unfinished Journey: A vista of uncharted territories, the allure of unconquered summits beckoning the daring, the realization that the journey is ongoing, with the destination forever elusive..jpeg'
Image for keyword 'Holistic Living: Embracing the richness of life's experiences, refusing to confine oneself to a single path, exploring all facets of existence with boundless curiosity and vigor.' saved as 'Holistic Living: Embracing the richness of life's experiences, refusing to confine oneself to a single path, exploring all facets of existence with boundless curiosity and vigor..jpeg'
Image for keyword 'Overcoming Fear: Scaling the peaks of challenges, navigating the valleys of uncertainty, overcoming the fear of redefining oneself and embracing the unknown with cou

## resize and crop

In [39]:

from PIL import Image

def resize_and_crop(image_path,desired_height = 720 ):
    
    img = Image.open(image_path)

#     desired_height = 720  # Adjust this to your desired height
    width, height = img.size
    ratio = height / desired_height
    new_width = int(width / ratio)

    resized_img = img.resize((new_width, desired_height))

# #     resized_image_path = "images/resized.jpg"
#     resized_img.save(image_path)

    
    # Open the image
    image = resized_img

    # Get the original width and height
    width, height = resized_img.size

    # Define the new desired width
    new_width = (9/16)*height  # Adjust this value as needed

    # Calculate the left and right crop positions to maintain aspect ratio
    left = int((width - new_width) / 2)
    right = left + new_width

    # Create the crop box using (left, top, right, bottom)
    crop_box = (left, 0, right, height)

    # Crop the image
    cropped_image = image.crop(crop_box)

    # Save the cropped image (optional)
    cropped_image.save(image_path)
    # Optionally display the resized image
    # resized_img.show()


time: 2.32 ms (started: 2024-07-31 16:35:41 +05:45)


## zoom

In [40]:
def Zoom(clip,mode='in',position='center',speed=1):
    fps = clip.fps
    duration = clip.duration
    total_frames = int(duration*fps)
    def main(getframe,t):
        frame = getframe(t)
        h,w = frame.shape[:2]
        i = t*fps
        if mode == 'out':
            i = total_frames-i
        zoom = 1+(i*((0.1*speed)/total_frames))
        positions = {'center':[(w-(w*zoom))/2,(h-(h*zoom))/2],
                     'left':[0,(h-(h*zoom))/2],
                     'right':[(w-(w*zoom)),(h-(h*zoom))/2],
                     'top':[(w-(w*zoom))/2,0],
                     'topleft':[0,0],
                     'topright':[(w-(w*zoom)),0],
                     'bottom':[(w-(w*zoom))/2,(h-(h*zoom))],
                     'bottomleft':[0,(h-(h*zoom))],
                     'bottomright':[(w-(w*zoom)),(h-(h*zoom))]}
        tx,ty = positions[position]
        M = np.array([[zoom,0,tx], [0,zoom,ty]])
        frame = cv2.warpAffine(frame,M,(w,h))
        return frame
    return clip.fl(main)

time: 2.4 ms (started: 2024-07-31 16:35:42 +05:45)


## crop and make video

Preset Choices are: ultrafast, superfast, veryfast, faster, fast, medium, slow, slower, veryslow, placebo. 

Video sizes may get big with ultrafast presets

https://stackoverflow.com/questions/56862176/why-is-my-implementation-of-creating-a-movie-in-moviepy-so-inefficient

video quality wont differ as written in the stack overflow. 

In [41]:

from moviepy.editor import VideoFileClip, ImageClip
import numpy as np

# from moviepy.editor import *
import cv2
# import numpy as np

for item in index_to_find_images:
    key, val = next(iter(item.items()))


    # Load the audio from another video file
    audio_clip = VideoFileClip('Video.mp4').subclip(val['start'], val['end']).audio


    img = 'images/' + val['keywords'] + '.jpeg'
    
    resize_and_crop(img)
    clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
    clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
    # Set the audio to the image clip
    clip = clip.set_audio(audio_clip)
    clip.write_videofile(f'subclips/{key}_{key}.mp4',preset='superfast')
    
    

Moviepy - Building video subclips/3_3.mp4.
MoviePy - Writing audio in 3_3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/3_3.mp4



Moviepy - Done !
Moviepy - video ready subclips/3_3.mp4
Moviepy - Building video subclips/7_7.mp4.
MoviePy - Writing audio in 7_7TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/7_7.mp4



Moviepy - Done !
Moviepy - video ready subclips/7_7.mp4
Moviepy - Building video subclips/11_11.mp4.
MoviePy - Writing audio in 11_11TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/11_11.mp4



Moviepy - Done !
Moviepy - video ready subclips/11_11.mp4
Moviepy - Building video subclips/14_14.mp4.
MoviePy - Writing audio in 14_14TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/14_14.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14.mp4
time: 14.9 s (started: 2024-07-31 16:35:56 +05:45)


In [42]:
len(interval_words) - 1

19

time: 4.38 ms (started: 2024-07-31 16:36:35 +05:45)


In [43]:
prev = 0

breaking_points = []
for item in index_to_find_images:
    key, val = next(iter(item.items()))
    
    breaking_points.append((prev,key-1, 0)) #start, end
    breaking_points.append((key,key, 1)) #images
    
    
    
    prev = key+1
    
if (breaking_points[-1][0],breaking_points[-1][0])  != (len(interval_words) - 1, len(interval_words) - 1):
    breaking_points.append((prev,len(interval_words) - 1, 0))

time: 2.77 ms (started: 2024-07-31 16:36:42 +05:45)


In [44]:
breaking_points

[(0, 2, 0),
 (3, 3, 1),
 (4, 6, 0),
 (7, 7, 1),
 (8, 10, 0),
 (11, 11, 1),
 (12, 13, 0),
 (14, 14, 1),
 (15, 19, 0)]

time: 3.52 ms (started: 2024-07-31 16:36:43 +05:45)


In [45]:
breaking_points

[(0, 2, 0),
 (3, 3, 1),
 (4, 6, 0),
 (7, 7, 1),
 (8, 10, 0),
 (11, 11, 1),
 (12, 13, 0),
 (14, 14, 1),
 (15, 19, 0)]

time: 5.03 ms (started: 2024-07-31 16:37:03 +05:45)


In [46]:
for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    
    
    if item[0] != item[1]:     
        print(first_start, second_end , 'gg' , breaking_points[index])
    
    
    else:
        
    #         clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
        prev_index_end = interval_words[breaking_points[index-1][1]]['end']
        current_end = interval_words[breaking_points[index+1][0]]['start']
        
        print(prev_index_end , current_end )
    

94.58000183105469 106.80000305175781 gg (0, 2, 0)
106.80000305175781 110.9800033569336
110.9800033569336 124.73999786376953 gg (4, 6, 0)
124.73999786376953 129.02000427246094
129.02000427246094 141.6199951171875 gg (8, 10, 0)
141.6199951171875 146.22000122070312
146.22000122070312 155.0 gg (12, 13, 0)
155.0 159.32000732421875
159.32000732421875 179.9199981689453 gg (15, 19, 0)
time: 3.16 ms (started: 2024-07-31 16:37:12 +05:45)


## crop whole video to portrait size

In [48]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

ffmpeg_extract_subclip("main.mp4", 0, 240, targetname="test1.mp4")

import subprocess

# Define the ffmpeg command
command = ["ffmpeg", "-i", "test1.mp4", "-filter:v", "crop=405:720", "-c:v", "libx264", "-crf", "22", "subclips/cropped.mp4"]

# Execute the command using subprocess.run
subprocess.run(command)

print("FFmpeg command executed successfully!")


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
FFmpeg command executed successfully!
time: 349 ms (started: 2024-07-31 16:39:39 +05:45)


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --en

In [50]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    

    
    video = VideoFileClip("subclips/cropped.mp4")
    
    print(index)
    
    
    if item[0] != item[1]:
        print(first_start)
        print(second_end)
#         ffmpeg_extract_subclip("subclips/cropped.mp4", first_start, second_end -2, targetname=f"subclips/{item[0]}_{item[1]}.mp4")
#         ffmpeg_extract_subclip("subclips/cropped.mp4", second_end -2, second_end, targetname=f"subclips/end_{item[0]}_{item[1]}.mp4")

        # Extract the subclips
        clip1 = video.subclip(first_start, second_end -2)
        clip2 = video.subclip(second_end -2, second_end)

        # Write the subclips to files (optional, for verification)
        clip1.write_videofile(f"subclips/{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')
        clip2.write_videofile(f"subclips/end_{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')

        
    else:

        
        # Load the audio from another video file
        
        img = 'images/' + interval_words[item[0]]['keywords'] + '.jpeg'

        resize_and_crop(img)
        
#         clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
        prev_index_end = interval_words[breaking_points[index-1][1]]['end']
        current_end = interval_words[breaking_points[index+1][0]]['start']
        
        print(prev_index_end,current_end )
        
        audio_clip = VideoFileClip('subclips/cropped.mp4').subclip(prev_index_end, current_end).audio
        
        clip = ImageClip(img).set_fps(30).set_duration(current_end - prev_index_end)
        clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
        # Set the audio to the image clip
        clip = clip.set_audio(audio_clip)

        
        clip.write_videofile(f'subclips/{item[0]}_{item[0]}.mp4',preset='superfast', codec='libx264', audio_codec='aac')

        

0
94.58000183105469
106.80000305175781
Moviepy - Building video subclips/0_2.mp4.
MoviePy - Writing audio in 0_2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/0_2.mp4



Moviepy - Done !
Moviepy - video ready subclips/0_2.mp4
Moviepy - Building video subclips/end_0_2.mp4.
MoviePy - Writing audio in end_0_2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_0_2.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_0_2.mp4
1
106.80000305175781 110.9800033569336
Moviepy - Building video subclips/3_3.mp4.
MoviePy - Writing audio in 3_3TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready subclips/14_14.mp4
8
159.32000732421875
179.9199981689453
Moviepy - Building video subclips/15_19.mp4.
MoviePy - Writing audio in 15_19TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/15_19.mp4



Moviepy - Done !
Moviepy - video ready subclips/15_19.mp4
Moviepy - Building video subclips/end_15_19.mp4.
MoviePy - Writing audio in end_15_19TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_15_19.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_15_19.mp4
time: 27.3 s (started: 2024-07-31 16:42:09 +05:45)


In [332]:
index_to_find_images

[{3: {'start': 107.94000244140625,
   'end': 112.94000244140625,
   'words': ['to',
    'serve',
    'in',
    'the',
    'Senate',
    'or',
    'the',
    'government',
    'Then',
    'you',
    'become',
    'a',
    'philosopher',
    "There's",
    'sort',
    'of',
    'this',
    'arc',
    'to',
    'life',
    'where'],
   'keywords': 'Path of Wisdom: The journey from youthful enthusiasm to statesmanship and philosophical contemplation, a tapestry of roles and responsibilities shaping a life of depth and insight.'}},
 {5: {'start': 118.72000122070312,
   'end': 123.72000122070312,
   'words': ['is',
    'for',
    'insects',
    'right',
    'So',
    'everyone',
    'should',
    'just',
    'be',
    'able',
    'to',
    'do',
    'everything',
    'And',
    'so',
    'I',
    "don't",
    'believe'],
   'keywords': 'Embracing Variety: A world where specialization gives way to exploration, where individuals are versatile like chameleons, adapting and thriving in diverse r

time: 7.84 ms (started: 2024-05-27 20:14:57 +05:45)


In [368]:
len(breaking_points)

15

time: 3.94 ms (started: 2024-05-27 21:24:20 +05:45)


In [398]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    

    
    video = VideoFileClip("subclips/cropped.mp4")
    
    print(index)

    if item[2] == 0:
        print(first_start)
        print(second_end)
#         ffmpeg_extract_subclip("subclips/cropped.mp4", first_start, second_end -2, targetname=f"subclips/{item[0]}_{item[1]}.mp4")
#         ffmpeg_extract_subclip("subclips/cropped.mp4", second_end -2, second_end, targetname=f"subclips/end_{item[0]}_{item[1]}.mp4")

        # Extract the subclips
        clip1 = video.subclip(first_start, second_end -3)
        clip2 = video.subclip(second_end -3, second_end)

        # Write the subclips to files (optional, for verification)
        clip1.write_videofile(f"subclips/{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')
        clip2.write_videofile(f"subclips/end_{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')

        
    else:
        
        # Load the audio from another video file
        
        img = 'images/' + interval_words[item[0]]['keywords'] + '.jpeg'

        resize_and_crop(img)
        
#         clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
        prev_index_end = interval_words[breaking_points[index-1][1]]['end']
    
        if index == len(breaking_points) - 1:
            current_end = interval_words[breaking_points[index][0]]['start']
        else:
            
            current_end = interval_words[breaking_points[index+1][0]]['start']
        
        print(prev_index_end,current_end )
        
        audio_clip = VideoFileClip('subclips/cropped.mp4').subclip(prev_index_end, current_end).audio
        
        clip = ImageClip(img).set_fps(30).set_duration(current_end - prev_index_end)
        clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
        # Set the audio to the image clip
        clip = clip.set_audio(audio_clip)

        
        clip.write_videofile(f'subclips/{item[0]}_{item[0]}.mp4',preset='superfast', codec='libx264', audio_codec='aac')


0
92.41999816894531
100.87999725341797
Moviepy - Building video subclips/0_1.mp4.
MoviePy - Writing audio in 0_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/0_1.mp4



Moviepy - Done !
Moviepy - video ready subclips/0_1.mp4
Moviepy - Building video subclips/end_0_1.mp4.
MoviePy - Writing audio in end_0_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_0_1.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_0_1.mp4
1
100.87999725341797 105.22000122070312
Moviepy - Building video subclips/2_2.mp4.
MoviePy - Writing audio in 2_2TEMP_MPY_wvf_snd.mp4


...

MoviePy - Done.
Moviepy - Writing video subclips/21_21.mp4



Moviepy - Done !


Moviepy - video ready subclips/21_21.mp4
Moviepy - Building video subclips/end_21_21.mp4.
MoviePy - Writing audio in end_21_21TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_21_21.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_21_21.mp4
time: 25.9 s (started: 2024-05-28 09:42:29 +05:45)


## apply transition

In [51]:
import moviepy.editor as mpe

def apply_transition(prev,nextt, transition, output, scale_point):

    clip1 = mpe.VideoFileClip(prev)
    clip2 = mpe.VideoFileClip(nextt)
    transition_clip = mpe.VideoFileClip(transition)

    # Apply mask to remove the green background from the transition clip
    masked_transition = transition_clip.fx(mpe.vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

    # Define the duration of the transition
    transition_duration = masked_transition.duration

    # Ensure the transition does not exceed the duration of the clips
    if transition_duration > clip1.duration or transition_duration > clip2.duration:
        raise ValueError("Transition duration exceeds the duration of one or both video clips.")

    # Calculate the start times for the transition
    transition_start_clip1 = clip1.duration - transition_duration + scale_point
    transition_start_clip2 = transition_duration

    # Set the start time for the masked transition clip
    masked_transition = masked_transition.set_start(transition_start_clip1)

    # Create the composite video with the transition
    composite_transition = mpe.CompositeVideoClip([
        clip1,
        masked_transition,
        clip2.set_start(clip1.duration)
#         clip2
    ]).set_duration(clip1.duration + clip2.duration)


    composite_transition.write_videofile(output, codec='libx264', 
                         audio_codec='aac', preset='superfast')

time: 2.82 ms (started: 2024-07-31 16:42:47 +05:45)


In [52]:
breaking_points

[(0, 2, 0),
 (3, 3, 1),
 (4, 6, 0),
 (7, 7, 1),
 (8, 10, 0),
 (11, 11, 1),
 (12, 13, 0),
 (14, 14, 1),
 (15, 19, 0)]

time: 4.97 ms (started: 2024-07-31 16:42:49 +05:45)


In [53]:
random.randint(1,7)

3

time: 3.04 ms (started: 2024-07-31 16:42:52 +05:45)


In [54]:
# transition_video = 'clips/lightleaks/checklight1.mp4'

transition_video_list = [('clips/Main_transition_clips/checklight1.mp4', 0.8), ('clips/Main_transition_clips/checklight1.mp4', 0.8), ('clips/Main_transition_clips/checklight4.mp4', 1), ('clips/Main_transition_clips/checklight4.mp4', 1), ('clips/Main_transition_clips/checklight5.mp4', 0.5), ('clips/Main_transition_clips/checklight6.mp4', 0.8), ('clips/Main_transition_clips/checklight6.mp4', 0.8)]

for index,item in enumerate(breaking_points):
    first = item[0]
    second = item[1]
    
    prev = index-1
    
    if item[2] == 1:
        random_index = random.randint(0,6)
        transition_video = transition_video_list[random_index][0]
        
        
        scale_point = transition_video_list[random_index][1]
        
        print(transition_video,scale_point )
        
        prev_video = f'subclips/end_{breaking_points[prev][0]}_{breaking_points[prev][1]}.mp4'
        nextt_video = f'subclips/{first}_{second}.mp4'
        output = f'subclips/{first}_{second}_new.mp4'

        print('Combinig ',prev_video, nextt_video)
        apply_transition(prev_video,nextt_video, transition_video, output,scale_point )
    
    

clips/Main_transition_clips/checklight6.mp4 0.8
Combinig  subclips/end_0_2.mp4 subclips/3_3.mp4
Moviepy - Building video subclips/3_3_new.mp4.
MoviePy - Writing audio in 3_3_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/3_3_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/3_3_new.mp4
clips/Main_transition_clips/checklight6.mp4 0.8
Combinig  subclips/end_4_6.mp4 subclips/7_7.mp4
Moviepy - Building video subclips/7_7_new.mp4.
MoviePy - Writing audio in 7_7_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/7_7_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/7_7_new.mp4
clips/Main_transition_clips/checklight6.mp4 0.8
Combinig  subclips/end_8_10.mp4 subclips/11_11.mp4
Moviepy - Building video subclips/11_11_new.mp4.
MoviePy - Writing audio in 11_11_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/11_11_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/11_11_new.mp4
clips/Main_transition_clips/checklight6.mp4 0.8
Combinig  subclips/end_12_13.mp4 subclips/14_14.mp4
Moviepy - Building video subclips/14_14_new.mp4.
MoviePy - Writing audio in 14_14_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/14_14_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14_new.mp4
time: 49.8 s (started: 2024-07-31 16:43:00 +05:45)


## Combine all the video with original video + transition + images in between

In [55]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# List of video file paths to be concatenated
video_files = []

for item in breaking_points:
    first = item[0]
    second = item[1]
    
    
    if item[2] == 1:
        video_files.append(f'subclips/{first}_{second}_new.mp4')
    else:
        video_files.append(f'subclips/{first}_{second}.mp4')



# Load each video file into a VideoFileClip object
clips = [VideoFileClip(video) for video in video_files]

# Concatenate the list of video clips into a single video
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final video file
final_clip.write_videofile("subclips/final_video_new.mp4", preset='superfast', codec='libx264', audio_codec='aac' )


Moviepy - Building video subclips/final_video_new.mp4.
MoviePy - Writing audio in final_video_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/final_video_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/final_video_new.mp4
time: 23.5 s (started: 2024-07-31 16:44:00 +05:45)


## add transcript

### whispeer the audio

In [66]:
from openai import OpenAI
client = OpenAI()

audio_file= open("subclips/final_video_new.mp4", "rb")
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

time: 13.2 s (started: 2024-07-31 16:56:22 +05:45)


In [56]:

import pickle
with open('shorttranscript.pkl', 'wb') as file:
    pickle.dump(transcript, file)

time: 10.3 ms (started: 2024-07-31 16:45:43 +05:45)


In [67]:
transcript.words

[{'word': 'So', 'start': 0.0, 'end': 0.30000001192092896},
 {'word': 'I', 'start': 0.30000001192092896, 'end': 0.6200000047683716},
 {'word': 'like', 'start': 0.6200000047683716, 'end': 0.9599999785423279},
 {'word': 'the', 'start': 0.9599999785423279, 'end': 1.440000057220459},
 {'word': 'model', 'start': 1.440000057220459, 'end': 1.7999999523162842},
 {'word': 'of', 'start': 1.7999999523162842, 'end': 2.299999952316284},
 {'word': 'life', 'start': 2.299999952316284, 'end': 2.380000114440918},
 {'word': 'that', 'start': 2.380000114440918, 'end': 2.640000104904175},
 {'word': 'the', 'start': 2.640000104904175, 'end': 3.0199999809265137},
 {'word': 'ancients', 'start': 3.0199999809265137, 'end': 3.240000009536743},
 {'word': 'had', 'start': 3.240000009536743, 'end': 3.5199999809265137},
 {'word': 'the', 'start': 3.7200000286102295, 'end': 3.9800000190734863},
 {'word': 'Greeks', 'start': 3.9800000190734863, 'end': 3.9800000190734863},
 {'word': 'the', 'start': 4.179999828338623, 'end': 

time: 26.3 ms (started: 2024-07-31 16:56:41 +05:45)


In [68]:
combined_transcript = []
new_transcript = transcript.words


for index, item in enumerate(new_transcript):
    start = item['start']
    end = item['end']
    duration = end - start
    
#     print(item['word'], duration)

    if index == 0: 
        continue
    if duration == 0:
        new_transcript[index-1]['end'] = new_transcript[index]['end']
        new_transcript[index-1]['word'] += " " + new_transcript[index]['word']
        new_transcript[index]['del'] = True
        
        
    

time: 2.58 ms (started: 2024-07-31 16:56:47 +05:45)


In [69]:
new_transcript

[{'word': 'So', 'start': 0.0, 'end': 0.30000001192092896},
 {'word': 'I', 'start': 0.30000001192092896, 'end': 0.6200000047683716},
 {'word': 'like', 'start': 0.6200000047683716, 'end': 0.9599999785423279},
 {'word': 'the', 'start': 0.9599999785423279, 'end': 1.440000057220459},
 {'word': 'model', 'start': 1.440000057220459, 'end': 1.7999999523162842},
 {'word': 'of', 'start': 1.7999999523162842, 'end': 2.299999952316284},
 {'word': 'life', 'start': 2.299999952316284, 'end': 2.380000114440918},
 {'word': 'that', 'start': 2.380000114440918, 'end': 2.640000104904175},
 {'word': 'the', 'start': 2.640000104904175, 'end': 3.0199999809265137},
 {'word': 'ancients', 'start': 3.0199999809265137, 'end': 3.240000009536743},
 {'word': 'had', 'start': 3.240000009536743, 'end': 3.5199999809265137},
 {'word': 'the Greeks',
  'start': 3.7200000286102295,
  'end': 3.9800000190734863},
 {'word': 'Greeks',
  'start': 3.9800000190734863,
  'end': 3.9800000190734863,
  'del': True},
 {'word': 'the Romans'

time: 30.9 ms (started: 2024-07-31 16:56:50 +05:45)


In [70]:
## delete the repeated one

combined_transcript = [item for item in new_transcript if not item.get('del', False) ]

time: 658 µs (started: 2024-07-31 16:56:55 +05:45)


## add subtitle

In [71]:
import sys

from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, AudioFileClip, CompositeAudioClip



def create_subtitle_clips(subtitles, videosize,fontsize=40, font='Helvetica-BoldOblique', color='white', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        start_time = subtitle['start']
        end_time = subtitle['end']
        duration = end_time - start_time

        video_width, video_height = videosize
        
        text_clip = TextClip(subtitle['word'], fontsize=fontsize, font=font, color=color ,method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 4 / 5 

        text_position = (subtitle_x_position, subtitle_y_position)                    
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips

time: 1.59 ms (started: 2024-07-31 16:56:58 +05:45)


In [402]:
# # Load video and SRT file
# video = VideoFileClip('subclips/final_video_new.mp4')
# subtitles = combined_transcript

# # begin,end= mp4filename.split(".mp4")
# output_video_file = "subclips/subtitled_video_final.mp4"

# print ("Output file name: ",output_video_file)

# # Create subtitle clips
# subtitle_clips = create_subtitle_clips(subtitles,video.size)

# # Add subtitles to the video
# final_video = CompositeVideoClip([video] + subtitle_clips)


# # Load background music
# background_music = AudioFileClip('')

# # Write output video file
# final_video.write_videofile(output_video_file,  preset='superfast', codec='libx264', audio_codec='aac')

Output file name:  subclips/subtitled_video_final.mp4


OSError: MoviePy error: the file  could not be found!
Please check that you entered the correct path.

time: 21.4 s (started: 2024-05-28 09:45:28 +05:45)


In [72]:

background_music = AudioFileClip('subclips/bg_music.mp4')

time: 262 ms (started: 2024-07-31 16:57:03 +05:45)


In [73]:
combined_transcript

[{'word': 'So', 'start': 0.0, 'end': 0.30000001192092896},
 {'word': 'I', 'start': 0.30000001192092896, 'end': 0.6200000047683716},
 {'word': 'like', 'start': 0.6200000047683716, 'end': 0.9599999785423279},
 {'word': 'the', 'start': 0.9599999785423279, 'end': 1.440000057220459},
 {'word': 'model', 'start': 1.440000057220459, 'end': 1.7999999523162842},
 {'word': 'of', 'start': 1.7999999523162842, 'end': 2.299999952316284},
 {'word': 'life', 'start': 2.299999952316284, 'end': 2.380000114440918},
 {'word': 'that', 'start': 2.380000114440918, 'end': 2.640000104904175},
 {'word': 'the', 'start': 2.640000104904175, 'end': 3.0199999809265137},
 {'word': 'ancients', 'start': 3.0199999809265137, 'end': 3.240000009536743},
 {'word': 'had', 'start': 3.240000009536743, 'end': 3.5199999809265137},
 {'word': 'the Greeks',
  'start': 3.7200000286102295,
  'end': 3.9800000190734863},
 {'word': 'the Romans', 'start': 4.179999828338623, 'end': 4.599999904632568},
 {'word': 'right', 'start': 4.699999809

time: 29.7 ms (started: 2024-07-31 16:57:06 +05:45)


In [74]:

# Load video and SRT file
video = VideoFileClip('subclips/final_video_new.mp4')

audio = AudioFileClip("Video.mp4")

# start_time = new_result[0]['transcript'][0]['start']
# end_time = new_result[0]['transcript'][-1]['end']

# audio_segment = audio.subclip(start_time, end_time)

subtitles = combined_transcript

output_video_file = "subclips/subtitled_video_final.mp4"
print("Output file name:", output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles, video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)

# Load background music
background_music = AudioFileClip('subclips/bg_music.mp4')

# Reduce the volume of the background music
background_music = background_music.volumex(0.2)  # Adjust volume as needed

# Combine original audio with background music
# We make the background music match the duration of the video
bg_music = background_music.set_duration(final_video.duration)
combined_audio = CompositeAudioClip([video.audio, bg_music])

# Set the combined audio as the audio for the final video
final_video = final_video.set_audio(combined_audio)

# Write output video file
final_video.write_videofile(output_video_file, preset='superfast', codec='libx264', audio_codec='aac')


Output file name: subclips/subtitled_video_final.mp4
Moviepy - Building video subclips/subtitled_video_final.mp4.
MoviePy - Writing audio in subtitled_video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/subtitled_video_final.mp4



Moviepy - Done !
Moviepy - video ready subclips/subtitled_video_final.mp4
time: 42.9 s (started: 2024-07-31 16:57:11 +05:45)


## testing cut and join

In [ ]:
128.9199981689453 164.66000366210938 gg (7, 13)
164.66000366210938 169.74000549316406

In [280]:
ffmpeg_extract_subclip("subclips/cropped.mp4", 160, 164.66000, targetname=f"subclips/test/1.mp4",codec='libx264', audio_codec='aac' )
ffmpeg_extract_subclip("subclips/cropped.mp4", 164.66000, 169.74000549316406, targetname=f"subclips/test/2.mp4", codec='libx264', audio_codec='aac')

TypeError: ffmpeg_extract_subclip() got an unexpected keyword argument 'codec'

time: 40.3 ms (started: 2024-05-25 12:50:01 +05:45)


In [275]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import concatenate_videoclips

# Cut the video into two parts using MoviePy
video = VideoFileClip("subclips/cropped.mp4")

# Extract the subclips
clip1 = video.subclip(160, 164.66000366210938)
clip2 = video.subclip(164.66000366210938, 169.74)

# Write the subclips to files (optional, for verification)
clip1.write_videofile("subclips/test/1.mp4", preset='superfast', codec='libx264', audio_codec='aac')
clip2.write_videofile("subclips/test/2.mp4", preset='superfast', codec='libx264', audio_codec='aac')

Moviepy - Building video subclips/test/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/1.mp4
Moviepy - Building video subclips/test/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/2.mp4
time: 2.75 s (started: 2024-05-25 12:45:12 +05:45)


In [ ]:

# Alternatively, skip writing subclips to files and concatenate directly



# Concatenate the subclips
final_clip = concatenate_videoclips([clip1, clip2], method="compose")

# Write the final combined video
final_clip.write_videofile("subclips/test/gg.mp4", preset='superfast', codec='libx264', audio_codec='aac')

In [278]:
## join


# Load each video file into a VideoFileClip object
clips = [VideoFileClip("subclips/test/1.mp4"), VideoFileClip("subclips/test/2.mp4")]


# Concatenate the list of video clips into a single video
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final video file
final_clip.write_videofile("subclips/test/gg1.mp4", preset='superfast', codec='libx264', audio_codec='aac' )


Moviepy - Building video subclips/test/gg1.mp4.
MoviePy - Writing audio in gg1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/gg1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/gg1.mp4
time: 2.87 s (started: 2024-05-25 12:46:50 +05:45)


In [197]:
start = new_result[0]['transcript'][0]['start']


92.41999816894531

In [198]:
end = new_result[0]['transcript'][-1]['end']

In [199]:
end

182.5800018310547

In [279]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import concatenate_videoclips

# Cut the video into two parts using MoviePy
video = VideoFileClip("subclips/cropped.mp4")

# Extract the subclips
clip1 = video.subclip(160, 164.66)
clip2 = video.subclip(164.66, 169.74)

# Write the subclips to files (optional, for verification)
clip1.write_videofile("subclips/test/1.mp4", preset='superfast', codec='libx264', audio_codec='aac')
clip2.write_videofile("subclips/test/2.mp4", preset='superfast', codec='libx264', audio_codec='aac')

# Alternatively, skip writing subclips to files and concatenate directly

# Concatenate the subclips
final_clip = concatenate_videoclips([clip1, clip2], method="compose")

# Write the final combined video
final_clip.write_videofile("subclips/test/gg2.mp4", preset='superfast', codec='libx264', audio_codec='aac')


Moviepy - Building video subclips/test/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/1.mp4
Moviepy - Building video subclips/test/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/2.mp4
Moviepy - Building video subclips/test/gg2.mp4.
MoviePy - Writing audio in gg2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/gg2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/gg2.mp4
time: 5.22 s (started: 2024-05-25 12:48:38 +05:45)
